In [1]:
#importando as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.core.display import HTML

In [2]:
#url montada para o primeiro acesso
url = input('Ente com o link da competição:')

qnt_paginas = 1
tempo = 3

#Efetua a consulta
req = requests.get(url, timeout = tempo)
content = req.content
url_open = BeautifulSoup(content, 'html.parser')

In [3]:
# --------------------------------------- EXPORTANDO DADOS DA COMPETIÇÃO --------------------------------------- #
#cria a lista de competições
list_competicao=[]

#pega o código da competição
id_competicao=url.replace('/all-games','')
conta_barras=id_competicao.count('/')
separa_barras=id_competicao.split('/')
cod_competicao =separa_barras[conta_barras]

#pega o código do país da competição
link_sem_competicao=id_competicao.replace(cod_competicao,'')
num_caracteres=len(link_sem_competicao)
cod_pais=link_sem_competicao[:num_caracteres-1]
conta_barras=cod_pais.count('/')
separa_barras=cod_pais.split('/')
cod_pais =separa_barras[conta_barras]

#localizando os ítens para alimentar a tabela
desc_competicao=url_open.find('p', attrs={'class': 'competition-title'})
desc_competicao=desc_competicao.text

#pega o local da competição
conta_espacos=desc_competicao.count(' ')
separa_espacos=desc_competicao.split(' ')
tempCompeticao =separa_espacos[conta_espacos]
localCompeticao=url_open.find('p', attrs={'class': 'competition-subtitle'})
localCompeticao=localCompeticao.text

#pega a temporada da competição
num_caracteres=len(desc_competicao)
num_caracteres_temp_competicao=len(tempCompeticao)
desc_competicao=desc_competicao[:num_caracteres-num_caracteres_temp_competicao-1]

#inserindo ítens na lista
list_competicao.append([cod_competicao, desc_competicao, localCompeticao, tempCompeticao, url])

#mostra a lista em dataframe
df = pd.DataFrame(list_competicao, columns = ['id_competicao','desc_competicao','local_competicao','temporada','link_competicao'])
display(df)

,id_competicao,desc_competicao,local_competicao,temporada,link_competicao
0,8pj1Q9y6oQb3k,Paulista A1,Brasil,2023,https://www.academiadasapostasbrasil.com/stats...


In [4]:
#limpa a lista
list_partidas = []
list_eventos = []
list_bd_eventos = []
list_atletas = []
list_estatisticas = []
list_times = []

#cria um contador das partidas
contador = 0
for i in range(qnt_paginas):
    resposta=requests.get(str(url)+'/page/'+str(i+1), timeout = tempo)
    conteudo=resposta.content
    site=BeautifulSoup(conteudo, 'html.parser')

    #localizando os ítens para alimentar a tabela
    tabela=site.find('table', attrs={'class': 'competition-rounds competition-half-padding'})
    partidas=tabela.findAll('tr')

    #fazendo o loop de todas as partidas
    print('linhas encontradas:', len(partidas))
    for partida in partidas:
        td_partida = partida.findAll('td')

        try:
            contador = contador + 1
            concluido=contador/len(partidas)*100
            print("%d%%" %concluido)
          
            ref=partida.findAll('a', attrs={'': ''})
            url_jg=ref[3]['href']

            resposta_e=requests.get(url_jg, timeout = 3)
            conteudo_e=resposta_e.content
            site_e=BeautifulSoup(conteudo_e, 'html.parser')
            pegaTimes=site_e.findAll('td', attrs={'stats-game-head-teamname hide-mobile'})
            
            #pega IdPartida da partida
            link_partida=url_jg
            url_live=link_partida.replace('/live','')
            conta_barras=url_live.count('/')
            separa_barras=url_live.split('/')
            IdPartida=separa_barras[conta_barras]

            time1=pegaTimes[0].find('a', attrs={'': ''})
            mandante=pegaTimes[0].text.strip()
            link_mandante=time1['href']
            img1=pegaTimes[0].find('img', attrs={'': ''})
            status=site_e.find('li', attrs={'gamestatus even'})
            placar=site_e.find('li', attrs={'class': 'f-score odd'})

            #pega IdMandante
            conta_barrasM=link_mandante.count('/')
            separa_barrasM=link_mandante.split('/')
            IdMandante=separa_barrasM[conta_barrasM]

            placar_extra=site_e.find('li', attrs={'class': 'et_pg gamehead'})
            p_extra=placar_extra.findAll('p', attrs={'': ''})
            try:
                prorrogacao=p_extra[0].text
            except:
                prorrogacao=''

            try:
                penaltis=p_extra[1].text
            except:
                penaltis=''

            img2=pegaTimes[1].find('img', attrs={'': ''})
            time2=pegaTimes[1].find('a', attrs={'': ''})
            link_visitante=time2['href']
            visitante=pegaTimes[1].text.strip()
            
            #pega IdVisitante
            conta_barrasV=link_visitante.count('/')
            separa_barrasV=link_visitante.split('/')
            IdVisitante=separa_barrasV[conta_barrasV]

            d_infor=site_e.find('td', attrs={'stats-game-head-date'})
            infor=d_infor.findAll('li', attrs={'class': 'gamehead'})
            data_hora=infor[1]
            liga=infor[2].find('a', attrs={'': ''})
            temporada=infor[3].find('a', attrs={'': ''})
            link_local=liga['href']
            
            #pega dados da competição
            link_liga=temporada['href']
            conta_barrasLiga=link_liga.count('/')
            separa_barrasLiga=link_liga.split('/')
            IdLiga=separa_barrasLiga[conta_barrasLiga]
            
            #pega dados do local da liga
            IdLocal=separa_barrasLiga[conta_barrasLiga-1]
            
            #pega dados da rodada
            rodadas=infor[4].find('a', attrs={'': ''})
            rodada=rodadas.text.replace('                                         - ',' - ')

            #pega estádio e cidade da partida
            try:
                table_estadio_cidade=site_e.find('table', attrs={'class': 'match-stadium stat-half-padding'})
                tr = table_estadio_cidade.findAll('tr')
                list_local = []
                for x in tr:
                    conta_itens=x.text.count(':')
                    separa_itens=x.text.split(':')
                    IdLiga=separa_itens[conta_itens]
                    IdLocal=separa_itens[conta_itens]
                    list_local.append(IdLocal)
                estadio = list_local[0]
                cidade = list_local[1]

                img_estadio = site_e.find('table', attrs={'class': 'match-stats-live-bg venue'})
                img_estadio = img_estadio.find('img')
                img_estadio = img_estadio['src']
            except:
                pass

            #adicionando elementos na lista
            list_partidas.append(
                [data_hora.text[:-8], data_hora.text[-5:], status.text.strip(), liga.text.strip(), temporada.text.strip(), 
                 IdLocal, IdLiga, rodada.strip(), IdPartida, mandante, IdMandante, img1['src'], placar.text.strip(), 
                 prorrogacao, penaltis, img2['src'], IdVisitante, visitante, cidade.strip(), estadio.strip(), img_estadio])
            list_times.append([IdMandante, mandante, img1['src'], link_mandante])
            list_times.append([IdVisitante, visitante, img2['src'], link_visitante])
            ##########################################################################################################
            table=site_e.find('table', attrs={'class': 'match_stats_center'})
            trs=table.findAll('tr', attrs={'': ''})

            for tr in trs:
                indicador=tr['class']
                estatA=tr.find('td', attrs={'class': 'stat_value_number_team_A'})
                estatB=tr.find('td', attrs={'class': 'stat_value_number_team_B'})
                list_estatisticas.append([IdPartida, IdMandante, estatA.text.strip(), indicador[0], estatB.text.strip(), IdVisitante])
            ##########################################################################################################

            #localizando os ítens para alimentar a tabela de eventos
            tabelas=site_e.findAll('tbody', attrs={'class': 'stat-quarts-padding'})
            
            for tabela in tabelas:
                eventos=tabela.findAll('tr', attrs={'': ''})

                for evento in eventos:
                    colunas=evento.findAll('td', attrs={'': ''})

                    if len(colunas) == 6:

                        #pega o id do evento
                        evento_id=evento['class']

                        #extrae os minutos dos eventos
                        minuto=evento.findAll('td', attrs={'class': 'match-sum-wd-minute'})            
                        minuto1=minuto[0].text
                        minuto2=minuto[1].text
                        
                        #extrae os atletas
                        atletas=evento.findAll('td', attrs={'class': 'match-sum-wd-description'})
                        atleta1=atletas[0].findAll('a', attrs={'': ''})
                        atleta2=atletas[1].findAll('a', attrs={'': ''})

                        try:
                            desc_atleta1_1=atleta1[0]['href']
                            desc_Atleta1_1=atleta1[0].text.strip()

                        except:
                            desc_atleta1_1=''
                            desc_Atleta1_1=''
                        
                        #pega dados do atleta
                        conta_barras_At1_1=desc_atleta1_1.count('/')
                        separa_barras_At1_1=desc_atleta1_1.split('/')
                        IdAtleta1_1=separa_barras_At1_1[conta_barras_At1_1]

                        try:
                            desc_atleta1_2=atleta1[1]['href']
                            desc_Atleta1_2=atleta1[1].text.strip()

                        except:
                            desc_atleta1_2=''
                            desc_Atleta1_2=''
                        
                        #pega dados do atleta
                        conta_barras_At1_2=desc_atleta1_2.count('/')
                        separa_barras_At1_2=desc_atleta1_2.split('/')
                        IdAtleta1_2=separa_barras_At1_2[conta_barras_At1_2]

                        try:
                            desc_atleta2_1=atleta2[0]['href']
                            desc_Atleta2_1=atleta2[0].text.strip()

                        except:
                            desc_atleta2_1=''
                            desc_Atleta2_1=''
                        
                        #pega dados do atleta
                        conta_barras_At2_1=desc_atleta2_1.count('/')
                        separa_barras_At2_1=desc_atleta2_1.split('/')
                        IdAtleta2_1=separa_barras_At2_1[conta_barras_At2_1]

                        try:
                            desc_atleta2_2=atleta2[1]['href']
                            desc_Atleta2_2=atleta2[1].text.strip()

                        except:
                            desc_atleta2_2=''
                            desc_Atleta2_2=''
                        
                        #pega dados do atleta
                        conta_barras_At2_2=desc_atleta2_2.count('/')
                        separa_barras_At2_2=desc_atleta2_2.split('/')
                        IdAtleta2_2=separa_barras_At2_2[conta_barras_At2_2]


                        #-----------------------------------------------------------------------------------------
                        try:
                            url_atleta1_1=desc_atleta1_1
                            resposta_atleta1_1=requests.get(url_atleta1_1, timeout = 3)
                            conteudo_atleta1_1=resposta_atleta1_1.content
                            site_atleta1_1=BeautifulSoup(conteudo_atleta1_1, 'html.parser')
                            ft_atleta1_1=site_atleta1_1.find('td', attrs={'class': 'hide-mobile'})
                            foto_atleta1_1=ft_atleta1_1.find('img', attrs={'': ''})
                            pais1_atleta1_1=site_atleta1_1.findAll('td', attrs={'class': 'stats-player-textleft'})

                            NomeAtleta=pais1_atleta1_1[3].text
                            PaisAtleta=pais1_atleta1_1[4].text

                            pais2_atleta1_1=pais1_atleta1_1[4].find('img', attrs={'': ''})
                            flag_pais_atleta=pais2_atleta1_1

                            #desc_atleta1_1
                            list_atletas.append(
                                [IdAtleta1_1,  foto_atleta1_1['alt'], foto_atleta1_1['src'], PaisAtleta.strip(), 
                                 flag_pais_atleta['src'], desc_Atleta1_1, IdMandante])
                        except:
                            pass
                        #-----------------------------------------------------------------------------------------
                        try:
                            url_atleta1_2=desc_atleta1_2
                            resposta_atleta1_2=requests.get(url_atleta1_2, timeout = 3)
                            conteudo_atleta1_2=resposta_atleta1_2.content
                            site_atleta1_2=BeautifulSoup(conteudo_atleta1_2, 'html.parser')
                            ft_atleta1_2=site_atleta1_2.find('td', attrs={'class': 'hide-mobile'})
                            foto_atleta1_2=ft_atleta1_2.find('img', attrs={'': ''})
                            pais1_atleta1_2=site_atleta1_2.findAll('td', attrs={'class': 'stats-player-textleft'})

                            NomeAtleta=pais1_atleta1_2[3].text
                            PaisAtleta=pais1_atleta1_2[4].text
  
                            pais2_atleta1_2=pais1_atleta1_2[4].find('img', attrs={'': ''})
                            flag_pais_atleta=pais2_atleta1_2

                            list_atletas.append(
                                [IdAtleta1_2, foto_atleta1_2['alt'], foto_atleta1_2['src'], PaisAtleta.strip(), 
                                 flag_pais_atleta['src'], desc_Atleta1_2, IdMandante])
                        except:
                            pass
                        #-----------------------------------------------------------------------------------------
                        try:
                            url_atleta2_1=desc_atleta2_1
                            resposta_atleta2_1=requests.get(url_atleta2_1, timeout = 3)
                            conteudo_atleta2_1=resposta_atleta2_1.content
                            site_atleta2_1=BeautifulSoup(conteudo_atleta2_1, 'html.parser')
                            ft_atleta2_1=site_atleta2_1.find('td', attrs={'class': 'hide-mobile'})
                            foto_atleta2_1=ft_atleta2_1.find('img', attrs={'': ''})
                            pais1_atleta2_1=site_atleta2_1.findAll('td', attrs={'class': 'stats-player-textleft'})

                            NomeAtleta=pais1_atleta2_1[3].text
                            PaisAtleta=pais1_atleta2_1[4].text

                            pais2_atleta2_1=pais1_atleta2_1[4].find('img', attrs={'': ''})
                            flag_pais_atleta=pais2_atleta2_1

                            list_atletas.append(
                                [IdAtleta2_1, foto_atleta2_1['alt'], foto_atleta2_1['src'], PaisAtleta.strip(), 
                                 flag_pais_atleta['src'], desc_Atleta2_1, IdVisitante])
                        except:
                            pass
                        #-----------------------------------------------------------------------------------------
                        try:
                            url_atleta2_2=desc_atleta2_2
                            resposta_atleta2_2=requests.get(url_atleta2_2, timeout = 3)
                            conteudo_atleta2_2=resposta_atleta2_2.content
                            site_atleta2_2=BeautifulSoup(conteudo_atleta2_2, 'html.parser')
                            ft_atleta2_2=site_atleta2_2.find('td', attrs={'class': 'hide-mobile'})
                            foto_atleta2_2=ft_atleta2_2.find('img', attrs={'': ''})
                            pais1_atleta2_2=site_atleta2_2.findAll('td', attrs={'class': 'stats-player-textleft'})

                            NomeAtleta=pais1_atleta2_2[3].text
                            PaisAtleta=pais1_atleta2_2[4].text

                            pais2_atleta2_2=pais1_atleta2_2[4].find('img', attrs={'': ''})
                            flag_pais_atleta=pais2_atleta2_2
                            
                            #desc_atleta2_2
                            list_atletas.append(
                                [IdAtleta2_2, foto_atleta2_2['alt'], foto_atleta2_2['src'], PaisAtleta.strip(), 
                                 flag_pais_atleta['src'], desc_Atleta2_2, IdVisitante])
                        except:
                            pass
                        #-----------------------------------------------------------------------------------------
                        #extrae os eventos
                        event=evento.findAll('td', attrs={'class': 'match-sum-wd-symbol'})
                        desc_event1=event[0].find('span', attrs={'': ''})
                        desc_event2=event[1].find('span', attrs={'': ''})
                        
                        try:
                            desc_evento1=desc_event1
                            desc_evento2=''

                            #adicionando elementos na lista
                            list_eventos.append(
                                [IdPartida, 
                                IdMandante, 
                                IdVisitante, 
                                evento_id[0].replace('event-', ''), 
                                minuto1.replace('\xa0',''), 
                                desc_evento1['title'], 
                                IdAtleta1_1,
                                IdAtleta1_2,
                                IdAtleta2_1,
                                IdAtleta2_2,
                                '', 
                                minuto2.replace('\xa0','')
                                ])

                            list_bd_eventos.append(
                                [desc_evento1['title']
                                ])
                        except:
                            pass
                        
                        try:
                            desc_evento1=''
                            desc_evento2=desc_event2

                            #adicionando elementos na lista
                            list_eventos.append(
                                [IdPartida, 
                                IdMandante, 
                                IdVisitante, 
                                evento_id[0].replace('event-', ''), 
                                minuto1.replace('\xa0',''), 
                                '', 
                                IdAtleta1_1,
                                IdAtleta1_2,
                                IdAtleta2_1,
                                IdAtleta2_2,  
                                desc_evento2['title'], 
                                minuto2.replace('\xa0','')
                                ])

                            list_bd_eventos.append(
                                [desc_evento2['title']
                                ])
                        except:
                            pass
        except:
            pass

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
100%


In [19]:
#criando um data frame com os ítens da lista
tb_partidas = pd.DataFrame(
    list_partidas, columns=['Data', 'Hora', 'Status', 'Competicao', 'Temporada', 'IdLocal', 'IdCompeticao', 
                            'Rodada', 'IdPartida', 'Mandante', 'IdMandante', 'ImgMandante', 'Placar', 'Prorrogacao',
                            'Penaltis', 'ImgVisitante', 'IdVisitante', 'Visitante', 'Cidade', 'Estadio', 'ImgEstadio'])
pd.options.display.max_colwidth = None  #largura das colunas
tb_partidas.to_csv('backup/tbJogos.csv', index = False) #salvando em csv
tb_partidas #imprime tabela

def path_to_image_html(path):
    return '<img src="'+ path + '" width="50" >'

partidas = tb_partidas[['Data', 'Hora', 'Rodada', 'Mandante', 'ImgMandante', 'Placar', 'ImgVisitante', 'Visitante', 
                        'Cidade', 'Estadio', 'ImgEstadio']]
partidas = partidas.rename(columns={'ImgMandante': 'img', 'ImgVisitante': 'img', 'ImgEstadio': 'img'})
HTML(partidas.to_html(escape=False, formatters=dict(img=path_to_image_html)))

,Data,Hora,Rodada,Mandante,img,Placar,img,Visitante,Cidade,Estadio,img
0,14 janeiro 2023,11:00,Época normal - Rodada 1,Inter Limeira,,0 - 3,,São Bernardo,Limeira,Estádio Major José Levy Sobrinho,
1,14 janeiro 2023,15:30,Época normal - Rodada 1,Portuguesa,,0 - 2,,Botafogo SP,São Paulo,Estádio Dr. Oswaldo Teixeira Duarte,
2,14 janeiro 2023,16:00,Época normal - Rodada 1,Santo André,,1 - 0,,Guarani,Santo André,Estádio Bruno José Daniel,
3,14 janeiro 2023,18:30,Época normal - Rodada 1,Palmeiras,,0 - 0,,São Bento,São Paulo,Allianz Parque,
4,14 janeiro 2023,20:30,Época normal - Rodada 1,Santos,,2 - 1,,Mirassol,Santos,Estádio Urbano Caldeira,
5,15 janeiro 2023,11:00,Época normal - Rodada 1,Água Santa,,1 - 3,,Ferroviária,Diadema,"Estadio Distrital Vila Alice (Diadema, São Paulo)",
6,15 janeiro 2023,16:00,Época normal - Rodada 1,RB Bragantino,,1 - 0,,Corinthians,Bragança Paulista,Estádio Nabi Abi Chedid,
7,15 janeiro 2023,18:30,Época normal - Rodada 1,São Paulo,,0 - 0,,Ituano,São Paulo,Estádio Nicolau Alayon,
8,17 janeiro 2023,15:30,Época normal - Rodada 2,São Bento,,2 - 1,,Santo André,Sorocaba,Estádio Municipal Walter Ribeiro,
9,17 janeiro 2023,19:30,Época normal - Rodada 2,Mirassol,,0 - 1,,Inter Limeira,Mirassol,Estádio José Maria de Campos Maia,


In [20]:
#remove duplicatas
list_equipes = []
[list_equipes.append(x) for x in list_times if x not in list_equipes]
tb_times = pd.DataFrame(list_equipes, columns=['id_equipe', 'abrev_equipe', 'img_equipe','link_pag_equipe'])

#acessando a pagina do time
list_itens = []
for time in range(len(list_equipes)):
    url_time = list_equipes[time][3]
    req = requests.get(url_time, timeout = tempo)
    content = req.content
    url_open = BeautifulSoup(content, 'html.parser')

    tabela=url_open.find('div', attrs={'class': 'boxed-title'})
    #print(tabela.text.strip())
    list_itens.append(tabela.text.strip())

#adiciona nova coluna com nome das equipes
tb_times['nome_equipe'] = list_itens
tb_times.to_csv('backup/times.csv', index = False)    #salvando em csv

def path_to_image_html(path):
    return '<img src="'+ path + '" width="50" >'
times_img = tb_times.rename(columns={'img_equipe': 'img'})
HTML(times_img.to_html(escape=False, formatters=dict(img=path_to_image_html)))

,id_equipe,abrev_equipe,img,link_pag_equipe,nome_equipe
0,3753,Inter Limeira,,https://www.academiadasapostasbrasil.com/stats/team/brasil/inter-de-limeira/3753,AA Internacional de Limeira
1,12627,São Bernardo,,https://www.academiadasapostasbrasil.com/stats/team/brasil/sao-bernardo/12627,São Bernardo FC
2,335,Portuguesa,,https://www.academiadasapostasbrasil.com/stats/team/brasil/portuguesa/335,Associacao Portuguesa de Desportos
3,2828,Botafogo SP,,https://www.academiadasapostasbrasil.com/stats/team/brasil/botafogo-sp/2828,Botafogo FC Ribeirão Preto
4,343,Santo André,,https://www.academiadasapostasbrasil.com/stats/team/brasil/santo-andre/343,EC Santo André
5,316,Guarani,,https://www.academiadasapostasbrasil.com/stats/team/brasil/guarani/316,Guarani FC de Campinas
6,310,Palmeiras,,https://www.academiadasapostasbrasil.com/stats/team/brasil/palmeiras/310,SE Palmeiras
7,6120,São Bento,,https://www.academiadasapostasbrasil.com/stats/team/brasil/sao-bento/6120,EC São Bento
8,319,Santos,,https://www.academiadasapostasbrasil.com/stats/team/brasil/santos/319,Santos FC Sao Paulo
9,10164,Mirassol,,https://www.academiadasapostasbrasil.com/stats/team/brasil/mirassol/10164,Mirassol Futebol Clube


In [21]:
#criando um data frame com os ítens da lista
tb_eventos = pd.DataFrame(
    list_eventos, columns=['IdPartida', 'IdMandante', 'IdVisitante', 'Id_Evento', 'Minuto1', 'Evento1', 
                           'Id_Atleta1-1', 'Id_Atleta1-2', 'Id_Atleta2-2', 'Id_Atleta2-1', 'Evento2', 'Minuto2'])
pd.options.display.max_colwidth = None  #largura das colunas
tb_eventos.to_csv('backup/tbEventos.csv', index = False)    #salvando em csv
tb_eventos  #mostrando a tabela

,IdPartida,IdMandante,IdVisitante,Id_Evento,Minuto1,Evento1,Id_Atleta1-1,Id_Atleta1-2,Id_Atleta2-2,Id_Atleta2-1,Evento2,Minuto2
0,1L9Qa6r5nYoPd,3753,12627,R2zmkNNk9zZ08,,,,,651364,,Goal,7'
1,1L9Qa6r5nYoPd,3753,12627,oJqmnJJ9LGmxN,18',Yellow card,64688,,,,,
2,1L9Qa6r5nYoPd,3753,12627,qxRQ5XXoy0m7E,,,,,75791,,Yellow card,20'
3,1L9Qa6r5nYoPd,3753,12627,kznmOzzGnKZxJ,46',Substitute in,541155,79817,,,,
4,1L9Qa6r5nYoPd,3753,12627,EGoYjMMyvRQqb,,,,,394736,75791,Substitute in,59'
...,...,...,...,...,...,...,...,...,...,...,...,...
1753,3DKQq7v4nZzEW,24139,310,2RBY0jzL8Rmw0,90'+1',Substitute in,368515,488865,,,,
1754,3DKQq7v4nZzEW,24139,310,kznmOzVyJ1ZxJ,90'+1',Substitute in,649669,230017,,,,
1755,3DKQq7v4nZzEW,24139,310,qxeZPrM77BQro,90'+2',Goal,9089,,,,,
1756,3DKQq7v4nZzEW,24139,310,qxRQ5XnyN0m7E,,,,,400791,242453,Substitute in,90'+2'


In [22]:
#criando um data frame com os ítens da lista

tb_atletas = pd.DataFrame(
    list_atletas, columns=['id_atleta', 'nome_atleta', 'foto_atleta', 'nacionalidade', 'bandeira_pais', 'apelido', 'id_equipe'])
pd.options.display.max_colwidth = None  #largura das colunas
tb_atletas = tb_atletas.drop_duplicates()   #remove duplicatas
tb_atletas.to_csv('backup/tbAtletas.csv', index = False)    #salvando em csv

tb_atletas = pd.merge(tb_atletas, tb_times[['id_equipe',  'img_equipe', 'abrev_equipe']], on=['id_equipe'], how='left')

def path_to_image_html(path):
    return '<img src="'+ path + '" width="50" >'

tb_atletas = tb_atletas.rename(columns={'foto_atleta': 'img', 'bandeira_pais': 'img', 'img_equipe': 'img'})
HTML(tb_atletas.to_html(escape=False, formatters=dict(img=path_to_image_html)))

,id_atleta,nome_atleta,img,nacionalidade,img,apelido,id_equipe,img,abrev_equipe
0,651364,C. Amaral Barletta de Almeida,,Brasil,,Chrystian Barletta,12627,,São Bernardo
1,64688,Douglas,,Brasil,,Douglas,3753,,Inter Limeira
2,75791,João Carlos Castro,,Brasil,,João Carlos,12627,,São Bernardo
3,541155,Djalma,,Brasil,,Djalma,3753,,Inter Limeira
4,79817,Claudinei,,Brasil,,Claudinei,3753,,Inter Limeira
5,394736,Felipe Marques,,Brasil,,Felipe Marques,12627,,São Bernardo
6,zjJmWNRWDZbBP,Bruno Menezes Cavalcante de Souza,,Brasil,,Menezes,3753,,Inter Limeira
7,531159,Chrigor,,Brasil,,Chrigor,3753,,Inter Limeira
8,567161,Denilson Carvalho da Silva,,Brasil,,Denilson,3753,,Inter Limeira
9,86260,Jonathas,,Brasil,,Jonathas,3753,,Inter Limeira


In [23]:
#criando um data frame com os ítens da lista
tb_bd_eventos = pd.DataFrame(
    list_bd_eventos, columns=['Evento'])
tb_bd_eventos = tb_bd_eventos.drop_duplicates() #remove duplicatas
pd.options.display.max_colwidth = None  #largura das colunas
tb_bd_eventos.to_csv('backup/tb_bd_Eventos.csv', index = False) #salvando em csv
tb_bd_eventos   #mostrando a tabela

,Evento
0,Goal
1,Yellow card
3,Substitute in
16,Own goal
52,Dudu
58,Penalty goal
76,Penalty missed
92,Red card
225,Yellow 2nd/RC


In [24]:
#criando um data frame com os ítens da lista
tb_estatisticas = pd.DataFrame(
    list_estatisticas, columns=['IdPartida', 'IdMandante', 'TimeA', 'Indicador', 'TimeB', 'IdVisitante'])
pd.options.display.max_colwidth = None  #largura das colunas
tb_estatisticas.to_csv('backup/tbEstatisticas.csv', index = False)  #salvando em csv

#mescla dados do mandante
tb_estatisticas = tb_estatisticas.rename(columns={'IdMandante': 'id_equipe'})
tb_estatisticas = pd.merge(tb_estatisticas, tb_times[['id_equipe', 'abrev_equipe', 'img_equipe']], on=['id_equipe'], how='left')

#volta o nome da coluna 'IdMandante'
tb_estatisticas = tb_estatisticas.rename(columns={'id_equipe': 'IdMandante'})

#mescla dados do visitante
tb_estatisticas = tb_estatisticas.rename(columns={'IdVisitante': 'id_equipe'})
tb_estatisticas = pd.merge(tb_estatisticas, tb_times[['id_equipe', 'abrev_equipe', 'img_equipe']], on=['id_equipe'], how='left')

#carregando os campos
tb_estatisticas = tb_estatisticas[['IdPartida', 'abrev_equipe_x', 'img_equipe_x', 'TimeA', 'Indicador', 'TimeB', 'img_equipe_y', 'abrev_equipe_y']]

#tb_estatisticas = tb_estatisticas[['IdPartida', 'desc_time_x', 'img_time', 'TimeA', 'Indicador', 'TimeB', 'IdVisitante']]
tb_estatisticas = tb_estatisticas.rename(columns={'img_equipe_x': 'img', 'img_equipe_y': 'img'})
#tb_estatisticas #mostrando a tabela

def path_to_image_html(path):
    return '<img src="'+ path + '" width="50" >'

HTML(tb_estatisticas.to_html(escape=False, formatters=dict(img=path_to_image_html)))

,IdPartida,abrev_equipe_x,img,TimeA,Indicador,TimeB,img,abrev_equipe_y
0,1L9Qa6r5nYoPd,Inter Limeira,,50%,possession,50%,,São Bernardo
1,1L9Qa6r5nYoPd,Inter Limeira,,,goals,3,,São Bernardo
2,1L9Qa6r5nYoPd,Inter Limeira,,3,offsides,,,São Bernardo
3,1L9Qa6r5nYoPd,Inter Limeira,,,yellow_red_cards,,,São Bernardo
4,1L9Qa6r5nYoPd,Inter Limeira,,1,yellow_cards,2,,São Bernardo
5,1L9Qa6r5nYoPd,Inter Limeira,,13,fouls,10,,São Bernardo
6,1L9Qa6r5nYoPd,Inter Limeira,,4,substitutions,5,,São Bernardo
7,1L9Qa6r5nYoPd,Inter Limeira,,5,corners,,,São Bernardo
8,1L9Qa6r5nYoPd,Inter Limeira,,,red_cards,,,São Bernardo
9,2RBY00glRYw0J,Portuguesa,,67%,possession,33%,,Botafogo SP
